Imports for the functions

In [30]:
import collections
import random
import numpy as np
import re

Open txt file that is saved in the same folder. Then read in reviews and assign entries starting with ones to positive reviews and those with -1 to negative reviews

In [68]:
usingSmoothing = True

with open('simple-food-reviews.txt') as reviewsDoc:
    reviews = [line.rstrip() for line in reviewsDoc]
    positiveReviews = reviews[0:9]
    negativeReviews = reviews[9:18]
    
    print(reviews)
    print(positiveReviews)
    #print(positiveReviews.shape)
    print(negativeReviews)    #print(negativeReviews.shape)   

['1 the food is lovely', '1 this is a great restaurant', '1 i really enjoyed my food', '1 i enjoyed the experience at the restaurant', '1 we had a lovely meal', '1 my food tasted great', '1 the service was great', '1 what a lovely restaurant', '1 the food the service and the restaurant was great', '-1 the service is terrible', '-1 the food tasted awful', '-1 this is a bad restaurant', '-1 the food was really bad', '-1 the service and the food was terrible', '-1 we had a terrible experience', '-1 avoid this restaurant', '-1 avoid the food', '-1 the meal was terrible']
['1 the food is lovely', '1 this is a great restaurant', '1 i really enjoyed my food', '1 i enjoyed the experience at the restaurant', '1 we had a lovely meal', '1 my food tasted great', '1 the service was great', '1 what a lovely restaurant', '1 the food the service and the restaurant was great']
['-1 the service is terrible', '-1 the food tasted awful', '-1 this is a bad restaurant', '-1 the food was really bad', '-1 the

Just a function so split he words and return a counter set for the words. The counter function creates a two-tuple where you have (the word : number of occurences)
It needs to be passed a split array otherwise if you give it just a string it breaks down to letters - be warned.

In [70]:
def counter_text(randomReview):
    word = randomReview.split(' ')
    return collections.Counter(word)

Generate 12 random numbers from 0-17 so that we can choose some reviews to use for training. Assign the positive reviews to random positives so that we can tell which reviews are positive, do the same for the random negative reviews. Finally choose the reviews which are not known whether positive or negative that we will test using. Store their actual value 1 or -1 in an array with the corresponding indices.

In [71]:
x = np.array(random.sample(range(0,17),12))
y = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

randomReview = np.empty((0))
randomPositives = np.empty((0))
randomNegatives = np.empty((0))
testReviews = np.empty((0))
testReviewsAns = np.empty((0))
posC = collections.Counter()
negC = collections.Counter()
numN = 0
for i in range(12):
    if x[i] < 9:
        word = reviews[x[i]].split('1 ')
        randomPositives = np.append(randomPositives, word[1])
        posC = posC + counter_text(word[1])
    elif x[i] > 8:
        word = reviews[x[i]].split('-1 ')
        randomNegatives = np.append(randomNegatives, word[1])
        negC = negC + counter_text(word[1])
for i in range(18):
    if i not in x:
        word = reviews[i].split(' ', 1)
        testReviews = np.append(testReviews, word[1])
        testReviewsAns = np.append(testReviewsAns, word[0])
        
labelCounts = np.array([len(randomPositives),len(randomNegatives)])
print(labelCounts)
print(posC)
print(negC)

[7 5]
Counter({'the': 7, 'food': 4, 'great': 3, 'lovely': 2, 'service': 2, 'restaurant': 2, 'was': 2, 'my': 2, 'i': 2, 'enjoyed': 2, 'is': 1, 'and': 1, 'tasted': 1, 'we': 1, 'had': 1, 'a': 1, 'meal': 1, 'experience': 1, 'at': 1, 'really': 1})
Counter({'the': 4, 'terrible': 3, 'service': 2, 'food': 2, 'avoid': 2, 'is': 1, 'we': 1, 'had': 1, 'a': 1, 'experience': 1, 'and': 1, 'was': 1, 'this': 1, 'restaurant': 1})


Now that we have them seperated we need to create a collection of words so that we can find the conditional probabilities. So let us have a list of the words and then their respective frequency - this will make it easy to add words should neither category contain them later on.

In [72]:
allC = posC + negC
itemsAll = list(allC.items())
wordsAll, numberAll = [list(c) for c in zip(*itemsAll)]
itemsNeg = list(negC.items())
wordsNeg, numberNeg = [list(c) for c in zip(*itemsNeg)]
itemsPos = list(posC.items())
wordsPos, numberPos = [list(c) for c in zip(*itemsPos)]
print(itemsAll)
print(wordsPos)
#print(numberPos)
print(wordsAll)
#print(numberAll)
print(wordsNeg)
#print(numberNeg)
#print(itemsNeg)
#print(itemsPos)

[('the', 11), ('food', 6), ('is', 2), ('lovely', 2), ('service', 4), ('and', 2), ('restaurant', 3), ('was', 3), ('great', 3), ('my', 2), ('tasted', 1), ('we', 2), ('had', 2), ('a', 2), ('meal', 1), ('i', 2), ('enjoyed', 2), ('experience', 2), ('at', 1), ('really', 1), ('terrible', 3), ('avoid', 2), ('this', 1)]
['the', 'food', 'is', 'lovely', 'service', 'and', 'restaurant', 'was', 'great', 'my', 'tasted', 'we', 'had', 'a', 'meal', 'i', 'enjoyed', 'experience', 'at', 'really']
['the', 'food', 'is', 'lovely', 'service', 'and', 'restaurant', 'was', 'great', 'my', 'tasted', 'we', 'had', 'a', 'meal', 'i', 'enjoyed', 'experience', 'at', 'really', 'terrible', 'avoid', 'this']
['the', 'service', 'is', 'terrible', 'we', 'had', 'a', 'experience', 'and', 'food', 'was', 'avoid', 'this', 'restaurant']


Declare and fill main_table

In [73]:
num_rows = len(wordsAll)
main_table = np.zeros(shape = (num_rows,2))
#print(main_table)
#print(main_table.shape)

Run this for non-laplace smoothing

In [79]:
numPosComments = 0
numNegComments = 0
if usingSmoothing:
    numPosComments = len(randomPositives)
    numNegComments = len(randomNegatives)

    wordsAll = np.array(wordsAll)
    wordsAll = np.unique(wordsAll)
    #print(wordsAll)
    #for each label
    for i in range(2):
        #for each word
        for j in range(num_rows):
            curr_value_prob = 1.0
            if i == 0:
                #Then positive comment
                count_of_word = posC[wordsAll[j]]
                if count_of_word == 0:
                    count_of_word = 1
                curr_value_prob = count_of_word/numPosComments
            else:
                #Then negative comment
                count_of_word = negC[wordsAll[j]]
                if count_of_word == 0:
                    count_of_word = 1
                curr_value_prob = count_of_word/numNegComments

            main_table[j][i] = curr_value_prob

    print(numPosComments)
    print(main_table)
else:
    numPosComments = len(randomPositives)
    numNegComments = len(randomNegatives)

    wordsAll = np.array(wordsAll)
    wordsAll = np.unique(wordsAll)
    #print(wordsAll)
    #for each label
    for i in range(2):
        #for each word
        for j in range(num_rows):
            curr_value_prob = 1.0
            if i == 0:
                #Then positive comment
                count_of_word = posC[wordsAll[j]]
                curr_value_prob = count_of_word/numPosComments
            else:
                #Then negative comment
                count_of_word = negC[wordsAll[j]]
                curr_value_prob = count_of_word/numNegComments

            main_table[j][i] = curr_value_prob

    print(numPosComments)
    print(main_table)


7
[[0.14285714 0.2       ]
 [0.14285714 0.2       ]
 [0.14285714 0.2       ]
 [0.14285714 0.4       ]
 [0.28571429 0.2       ]
 [0.14285714 0.2       ]
 [0.57142857 0.4       ]
 [0.42857143 0.2       ]
 [0.14285714 0.2       ]
 [0.28571429 0.2       ]
 [0.14285714 0.2       ]
 [0.28571429 0.2       ]
 [0.14285714 0.2       ]
 [0.28571429 0.2       ]
 [0.14285714 0.2       ]
 [0.28571429 0.2       ]
 [0.28571429 0.4       ]
 [0.14285714 0.2       ]
 [0.14285714 0.6       ]
 [1.         0.8       ]
 [0.14285714 0.2       ]
 [0.28571429 0.2       ]
 [0.14285714 0.2       ]]


Predict function:

In [88]:
def predict_value(data_point):
    #data_point is vector(64) with pixels
    numerator_values = np.zeros(2)
    denominator = 0.0
    for i in range(2):
        curr_total = 1.0
        for j in range(num_rows):
            currWord = wordsAll[j]
            
            #currWordIndex = np.where(wordsAll == currWord)[0][0]
            if currWord not in data_point:
                #pixel is off
                curr_total = curr_total*(1-main_table[j][i])
            else:
                curr_total = curr_total*(main_table[j][i])
                
        numerator_values[i] = curr_total*(labelCounts[i]/12)
        denominator = denominator + numerator_values[i]
    
    #For each label get the prob
    largestProb = -1
    largestProbLabelNum = -1
    for labelNum in range(2):
        curr_prob = numerator_values[labelNum]/denominator
        
        #print(labelNum,curr_prob, denominator)
        
        if curr_prob > largestProb:
            largestProb = curr_prob
            largestProbLabelNum = labelNum
        
    
    if largestProbLabelNum == 0:
        return '1'
    elif largestProbLabelNum == 1:
        return '-1'

Test function:

In [89]:
print(testReviews)
print(testReviewsAns)

['this is a great restaurant' 'what a lovely restaurant'
 'the food tasted awful' 'this is a bad restaurant'
 'the food was really bad' 'the meal was terrible']
['1' '1' '-1' '-1' '-1' '-1']


In [90]:
total_num_tests = 6
total_num_correct = 0
#Predict each data_point
for i in range(len(testReviews)):
    test_dataPoint = testReviews[i].split(' ')
    test_dataPointAnswer = testReviewsAns[i]
    test_predict_answer = predict_value(test_dataPoint)
    if test_dataPointAnswer == test_predict_answer:
        total_num_correct = total_num_correct + 1

accuracy = total_num_correct/total_num_tests
print(accuracy)

0 0.0 1.2666373951979533e-07
1 1.0 1.2666373951979533e-07
0 0.0 2.0266198323167253e-06
1 1.0 2.0266198323167253e-06
0 0.8244034854672995 0.0004924306901855442
1 0.17559651453270045 0.0004924306901855442
0 0.0 5.066549580791813e-07
1 1.0 5.066549580791813e-07
0 0.8825160113199567 0.0001840019091473906
1 0.11748398868004338 0.0001840019091473906
0 0.2944440889990886 6.893695484260294e-05
1 0.7055559110009115 6.893695484260294e-05
0.3333333333333333


Calculate the probabilities of each probPos and probNeg. Could easily be done in one for loop. So if want to improve then just do it in one for loop instead. This should conclude the training portion since we have the required probabilities. I chose to do this function here so that we can use it in the question that does not do laplacian smoothing. It can be used easily then.
If we were going to select a piece of data that would confuse it then all we need to do is from the random set of reviews use the set function and then use the disjoint of the random review to check if it is not contained - since any element part of the random review that our model has not seen would give a 0 probability for both p(1) and p(-1) for the reviews actual value.

In [ ]:
probPos = list()
probNeg = list()
size = len(wordsAll)
for i in range(size):
    try:
        index = wordsPos.index(wordsAll[i])
        if index >= 0:
            probPos.append(numberPos[index]/numberAll[i])
    except:
        probPos.append(0)
for i in range(size):
    try:
        index = wordsNeg.index(wordsAll[i])
        if index >= 0:
            probNeg.append(numberNeg[index]/numberAll[i])
    except:
        probNeg.append(0)

        
print(probPos)
print(probNeg)

                                                                        ##CURENTLY NOT WORKING##

Now we need to take a sentence and calculate its most likely category, something is wrong here, definitely check to see what it is however feel free to check to see if the rest is working if you think I might have made a mistake. I tried to do a laplacian smoothing here where i checked to see if the word was contained in all before seen words and then to add an instance to each. It might be easier in this scenario to do a 'on the fly' check of each word, its index, number of times it occurs (since the list zip is in order) then we can always readily update however this will lead to slower computation speeds since we will be calculating that probability always. It might be easier for us though. Do what you feel would be easier or faster to complete the lab.

In [ ]:
def calcLikely()
    return(probability)

Just a quick run through of all the random reviews so that we can calculate the probability of each as required. Shows the probability of each and then what the actual value was.

In [ ]:
testReviewsSize = len(testReviews)
numberOfPositives = len(randomPositives)
numberOfNegatives = len(randomNegatives)
#print(numberOfPositives)
#print(numberOfNegatives)
for i in range(testReviewsSize):
    probabilities = np.array(calcLikely(testReviews[i], probPos, probNeg, wordsAll, wordsNeg, wordsPos, numberNeg, numberPos, numberAll, numberOfPositives, numberOfNegatives))
    print("probability of 1 is " + str(probabilities[0]))
    print("probability of -1 is " + str(probabilities[1]))
    print("Answer is actually " + str(testReviewsAns[i]))
    #print(np.argmax(probabilities))
    
print("finished")